In [1]:
import numpy as np

DIMENSIONS = (36, 36)

## Turn asteroid belt into 2D numpy array in which 1 represents an asteroid

In [2]:
with open("./input.txt") as f:
    asteroid_belt = f.read().replace("\n", "")

asteroid_belt = asteroid_belt.replace(".", "0").replace("#", "1")
asteroid_belt = np.array(list(map(int, list(asteroid_belt))))
asteroid_belt.shape = (36, 36)
asteroid_belt = asteroid_belt.transpose()

## Store (x, y) of asteroid locations in an list

In [3]:
asteroid_locs = [
    (x, y) for x in range(36)
    for y in range(36)
    if asteroid_belt[x, y] == 1
]

In [4]:
from functools import partial
from typing import Tuple

round_to_five_decimal = partial(round, ndigits=5)

def get_unit_vector_between_points(
    point1: Tuple[float, float], point2: Tuple[float, float]
) -> Tuple[float, float]:
    """Returns a vector of unit length between two points, rounded to 5 decimal places"""
    point_diff = np.array(point2) - np.array(point1)
    unit_vec = point_diff / np.linalg.norm(point_diff)

    return tuple(map(round_to_five_decimal, unit_vec))

## Determine how many asteroids are reachable from a given asteroid

Compute a unit vector to each asteroid and store the set of unit vectors to other asteroids. Because sets have unique elements, two identical unit vectors pointing to two different asteroids will not be counted twice.

In [5]:
reachable_asteroids = np.zeros((36, 36))

for asteroid in asteroid_locs:
    # get list of all asteroids other than current one
    other_asteroids = [other_asteroid for other_asteroid in asteroid_locs if asteroid != other_asteroid]
    unit_vector_to_asteroid = set()

    unit_vector_to_asteroid = set([
        get_unit_vector_between_points(asteroid, other_asteroid)
        for other_asteroid in other_asteroids
    ])

    reachable_asteroids[asteroid] = len(unit_vector_to_asteroid)

int(reachable_asteroids.max())

276